# TED Talk Transcription Artifacts

In this notebook we explore the various features of the TED talks documents that are a product of their being transcripts of oral performances. The transcription process itself is not entirely clear, since it is the product of volunteers. In practice, this means different volunteers, or groups of volunteers, transcribe the same or similar performance features differently. The goal of this notebook is to survey possible feature artifaces and to determine what, if any, of these artifacts need to be addressed when it comes to processing the documents for analysis.

First, we load our functions and then our documents:

In [1]:
# IMPORTS
import re 
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk import tokenize, text

In [2]:
# Loading the data in a gendered partitioned fashion: 
df_male = pd.read_csv('talks_male.csv', index_col='Talk_ID')
df_female = pd.read_csv('talks_female.csv', index_col='Talk_ID')
df_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')

df_all = pd.concat([df_male, df_female, df_nog])

texts = df_all.text.tolist()

print(f"From our {df_all.shape[0]} x {df_all.shape[1]} CSV, \
we have a list of {len(texts)} talks.")

From our 992 x 14 CSV, we have a list of 992 talks.


Drawing upon some of the early work in the *Terms* notebook, we run Sci-Kit Learn's `CountVectorizer` with only one parameter: for a word to be counted, it must occur in at least two documents.

In [3]:
# Default vectorizer = lowercase, remove punctuation, 
# tokens > 2 char, split contractions, no stopwords
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(texts)
X.shape

(992, 22433)

With the words in an array, we can inspect various segments of the word count. Inside this notebook, we use the `head()` method which with 20 entries, begins to suggest that there a lot of numbers to be taken into consideration. For a fuller examination, we also save the complete count to a CSV. (Here commented out so that it doesn't run every time the notebook runs.)

In [13]:
df_all_words = pd.DataFrame(X.toarray(), 
                      columns = vectorizer.get_feature_names_out())

wc_all = df_all_words.sum()
wc_all.head(20)
# wc_all.to_csv('../output/gender-part-word_counts.csv')

00          4
000      1187
000th       6
01          3
02          2
06          5
07          2
10        981
100       545
1000        5
100th       5
101        13
102         3
103         3
105        10
109         3
10th       20
11        223
110        10
111         3
dtype: int64

Having done that, and scrolling down to the 800th row before beginning to see letters forming something like words, we note that there are a lot of ways to spell out*ah*, which led us to ask our first question.

## Pause Fillers and Interjections: Variations on "ah"

The words from a search for "ah" were: 

    aaaah, aah, ah, ahh, ahhh
    
The words from a search for "aa" were: 

    aa, aaa, aaaa, aaaaa, aaaah, aah
    
Code used:

```python
for match in wc_all.index:
    if "aa" in match:
        print(match)
```

In [5]:
ahs_list = ["aa", "aaa", "aaaa", "aaaaa", "aah", 
            "aaaah", "aah", "ah", "ahh", "ahhh" ] 

ahs = wc_all.filter(items = ahs_list, axis=0)
ahs.head(len(ahs_list))

aa        11
aaa        6
aah       10
aaaah      2
aah       10
ah       116
ahh        6
dtype: int64

In [6]:
alltexts = ' '.join(texts).lower()
t = tokenize.WhitespaceTokenizer()
corpse = text.Text(t.tokenize(alltexts))
print(alltexts[0:100])

  thank you so much, chris. and it's truly a great honor to have the opportunity to come to this sta


In [7]:
corpse.concordance("ahh")

no matches


<div class="alert alert-block alert-warning">
There appears to be a tokenizer mis-match between SciKit-Learn, from which the list of <b>ah</b>s above is drawn and the NLTK tokenizer which is not finding the same tokens. 
</div>

### Word Count - Sorted

While we have a series with a word count, it's easy enough to sort the list to get a quick overview of the total count for the words in the corpus:

In [8]:
wc_sorted = wc_all.sort_values(ascending=False, inplace=False)
wc_sorted.head(10)

the     93853
and     67710
to      57089
of      52313
that    44087
it      35339
in      34728
you     34162
we      30407
is      28569
dtype: int64

## Performatives and Other Parenthetical Expressions <a class="anchor" id="performatives"></a>

Earlier explorations of the corpus revealed something we knew but had not realized could affect our work: some TED talks are not talks but musical performances. Generally, the text of such performances are rather short. Using an arbitrary length of `500` characters, we can see what these texts look like:

In [9]:
shorts = [ texts.index(text) for text in texts if len(text) < 500 ]
print(shorts)

[84, 183, 297, 388, 602, 983, 991]


In [10]:
print(f"{shorts[0]}:\n{texts[shorts[0]]}\n\n{shorts[1]}:\n{texts[shorts[1]]}")

84:
  (Applause)    (Music)    (Applause)  

183:
  Let's just get started here.    Okay, just a moment.    (Whirring)    All right. (Laughter) Oh, sorry.    (Music) (Beatboxing)    Thank you.    (Applause)  


As the two examples above reveal, some talks are not talks at all and do not actually contain any text, except for the performatives, while other talks contain non-textual materials, some of which is part of the performance and some of which is the audience's response to the performance. 

When it comes time to process words in a text, we think the best bet is to remove the performatives. We do think, however, that having them means we can possibly explore sentiment using `(Applause)` and `(Laughter)` as contextual valuations.

For now, we will need some regex to remove the parentheses and their contents from our texts. An examination of `113` above reveals that it is only three parenthetical expressions:

    (Applause)    (Music)    (Applause)

We need a sample text that is a mix, and so we will use `183` from above:

> Let's just get started here.    Okay, just a moment.    (Whirring)    All right. (Laughter) Oh, sorry.    (Music) (Beatboxing)    Thank you.    (Applause)

A quick check using two regexes give us one list without the parentheses and one with:

In [11]:
print(re.findall(r'(?<=\().*?(?=\))', texts[183]))
print(re.findall(r'\([^)]*\)', texts[183]))

['Whirring', 'Laughter', 'Music', 'Beatboxing', 'Applause']
['(Whirring)', '(Laughter)', '(Music)', '(Beatboxing)', '(Applause)']


We can also use `CountVectorizer` to inventory all the parenthetical expressions in the corpus to see if we are missing anything. 

First, we test is on a known text:

In [12]:
vec = CountVectorizer(token_pattern = r'(?<=\().*?(?=\))')
X183 = vec.fit_transform(texts[183:184])
df183 = pd.DataFrame(X183.toarray(), columns=vec.get_feature_names_out())
df.index = [ texts.index(text) for text in texts[183:184] ]
df.head()

NameError: name 'df' is not defined

In [ ]:
parentheticals = vec.fit_transform(texts)
parentheticals.shape

In [ ]:
df_parens = pd.DataFrame(parentheticals.toarray(), 
                         columns=vec.get_feature_names_out())
df_parens.index = [ texts.index(text) for text in texts ]
df_parens.head()

With all the parenthetical expressions captured, we can determine the most frequent, take a look at their numbers, and then decide what's the best path:

In [ ]:
sums = df_parens.sum(axis = 0)
sums.sort_values(ascending=False).head(25)

Based on these results, the top 20 parentheticals could be inserted into a stopword list and we would remove, in the case of the top 4 especially, words or clauses that might affect results.

The code below can be included in `CountVectorizer` as a preprocessor:

In [ ]:
# A Refined Preprocessor --
# This one removes two-word phrases/clauses

parentheticals = [ "\(laughter\)", "\(applause\)", "\(music\)",  
                  "\(video\)", "\(laughs\)", "\(applause ends\)", 
                  "\(audio\)", "\(singing\)", "\(music ends\)", 
                  "\(cheers\)", "\(cheering\)", "\(recording\)", 
                  "\(beatboxing\)", "\(audience\)", "\(guitar strum\)", 
                  "\(clicks metronome\)", "\(sighs\)", "\(guitar\)", 
                  "\(marimba sounds\)", "\(drum sounds\)" ]

def remove_parentheticals(text):
    global parentheticals
    new_text = text
    for rgx_match in parentheticals:
        new_text = re.sub(rgx_match, ' ', new_text.lower(), 
                          flags=re.IGNORECASE)
    return new_text

A quick test of the function:

In [ ]:
test = """Laughter is the best medicine. (Laughter) 
Hold your applause; I'm not done yet. (Applause ends)"""

print(remove_parentheticals(test))

In [ ]:
the_vec = CountVectorizer( preprocessor = remove_parentheticals,
                          max_df = 0.9, min_df = 2 )
the_X = the_vec.fit_transform(texts)
the_X.shape

## Numbers

One of the dimensions of the corpus that arises out of a hand inspection of the terms is the frequency with which some numbers appear. The follow table captures the top ten numbers:

| TERM | FREQUENCY |
|------|-----------|
| 000  | 2098 |
| 10   | 1691 |
|  20  | 1107 |
| 100  |  902 |
|  30  |  827 |
|  50  |  784 |
|  15  |  659 | 
|  40  |  494 |
|  12  |  460 | 
|  25  |  410 |

Other frequently occurring numbers: 60, 500, 200, 11, 18, 80, 14 (241 times!). 

In order to examine the appearance of the numbers in context, we make a giant string out of the list of strings, `texts`: in which text a number appears is less important than its immediate context. 

In [ ]:
onetext = nltk.Text('\n'.join(texts).split())
# And here's what an NLTK text object looks like: a list of words, really
print(onetext[0:10])

In [ ]:
onetext.concordance("000")

In [ ]:
onetext.concordance("10")

In [ ]:
onetext.concordance("40")

A couple of things to note here:

First, there is a discrepancy in the count between `sklearn` and the NLTK: the former counted 2098 occurrences of `000`, the latter none. In all the counts that follow, there is a similar mismatch:

| TERM | `sklearn` | `nltk` |
|------|-----------|--------|
| 000  | 2098 | "no match" |
|  10  | 1691 | 1216 |
|  20  | 1107 | 879 |
| 100  |  902 | 647 |
|  30  |  827 | 650 |
|  50  |  784 | 594 | 
|  15  |  659 | 512 | 
|  40  |  494 | 387 | 
| ...               | 
|  14  |  241 | 148 | 

I don't have a ready explanation for this.

Second, the frequency of some numbers are readily explained:

* Round numbers like 10, 20, 30, 50, and 100 are approximations -- though it would be interesting to explore how often they are attached to large scalars like "thousand" or million." 
* Some numbers seem to represent alternate ways of counting: 25 reagularly stands in for "one-quarter" -- though not as often as we might imagine -- and 18 is regularly paired with *month* as a more precise way to say " a year and a half."
* There are some numbers, like 11 and 14 which seem to have power all their own, perhaps tied to particular ages in humans. 

Next up is some code to explore the most common occurring words with these numbers.

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

All my searches for "collocations with specific words" took me to the NLTK, which means, so far as I can tell, generating all the bigrams and then filtering to get the one(s) you want. This seems backwards to me: wouldn't it be faster simply to find the word and then what comes after it? I'll take a look at regex for this later.

In [ ]:
## Bigrams
finder = BigramCollocationFinder.from_words(onetext)

In [ ]:
## Here's the filter operation:
the_number = lambda *w: '14' not in w
# only bigrams that appear 3+ times
finder.apply_freq_filter(3)
# only bigrams that contain the number
finder.apply_ngram_filter(the_number)
# return the 10 n-grams with the highest PMI
print(finder.nbest(bigram_measures.likelihood_ratio, 10))

This does not return a count. *Oi!*

### Trial 2

In [ ]:
the_one = nltk.Text(re.sub("[^a-zA-Z0-9']"," ",'\n'.join(texts)).lower().split())
# And here's what an NLTK text object looks like: a list of words, really
print(the_one[0:10])

In [ ]:
the_one.concordance("40")

Well, there's the missing `000`! It's in the idiomatic transcription practices of TED wherein a number like "sixty thousand" is rendered as "60,000." 

One thing we know now: reporting large numbers is a part of TED talks.

**TO DO**: How to keep the comma marker between numbers? (Or should we just look to 000 as a possible collocate with the other numbers?) One solution from the [Regex Cookbook][]:

```python
\b[0-9]{1,3}(,[0-9]{3})*(\.[0-9]+)?\b|\.[0-9]+\b
```

[Regex Cookbook]: https://www.oreilly.com/library/view/regular-expressions-cookbook/9781449327453/ch06s11.html